This code combines the two different flowline datasets into one. One only had end points of an flowline and the other had the entire flowline. We combined them using a spatial join and asserted that the operator name is the same, with a max 50 meter buffer. 

## Setup


In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiLineString, LineString,MultiPolygon, MultiPoint
from shapely.ops import nearest_points
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')
pd.options.display.max_columns = None

/var/folders/p3/gs65vvgs1wd90mnxb3q3hvy80000gp/T/ipykernel_1148/3583532370.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Load Data
flowlines_gdf = gpd.read_file('flowlines.geojson')
crudeoil_offlocation_gdf = gpd.read_file('crudeoil_offlocation.geojson')

In [5]:
# Check size
print(flowlines_gdf.shape)
print(crudeoil_offlocation_gdf.shape)

(21942, 19)
(259979, 8)


In [6]:
flowlines_gdf.head(2)

,LOCATION_ID,FLOWLINEID,STARTLOCATIONID,FLOWLINEACTION,ENTIRELINEREMOVED,ACTIONDESCRIPTION,RECEIVE_DATE,OPERATOR_NUM,COMPANY_NAME,LOCATIONTYPE,ENDLAT,ENDLONG,STARTLAT,STARTLONG,PIPEMATERIAL,BEDDINGMATERIAL,TYPEOFFLUIDTRANS,MAXOPPRESSURE,geometry
0,331872.0,485633.0,NaN,Registration,None,None,2023-10-30T08:10:52.119999,10673,GADECO LLC ...,Well Site,NaN,NaN,NaN,NaN,HDPE,Native Materials,Produced Water,NaN,None
1,470443.0,470446.0,NaN,None,None,The flowline serving the Emerson 3-29J (05-123...,2023-10-24T08:33:49.223000,10633,CRESTONE PEAK RESOURCES OPERATING LLC ...,Production Facilities,40.109444,-104.909686,40.105743,-104.90986,Carbon Steel,Native Materials,Multiphase,325.0,"LINESTRING (507682.442 4439497.658, 507696.855..."


In [7]:
crudeoil_offlocation_gdf.head(2)

,Operator,Fluid,Material,Diam_in,Status,Length_ft,SHAPE_Length,geometry
0,EVERGREEN NATURAL RESOURCES LLC,Gas,polly,4.0,Active,2277.71,693.972162,"MULTILINESTRING ((524642.670 4117088.796, 5246..."
1,NOBLE ENERGY INC,Multiphase,Carbon Steel,3.0,Abandoned,651.58,198.525215,"MULTILINESTRING ((527997.281 4463899.920, 5281..."


In [8]:
# Check if CRS is the same for both files
if flowlines_gdf.crs != crudeoil_offlocation_gdf.crs:
    flowlines_gdf = flowlines_gdf.to_crs(crudeoil_offlocation_gdf.crs)

## Spatial Join

In [ ]:
# Spatial join to find intersecting geometries
# Add attributes from flowlines to matching entries in crude_oil_gdf
matches_gdf = gpd.sjoin(crudeoil_offlocation_gdf, flowlines_gdf, how="inner", op='intersects')
matches_gdf.shape

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


(108291, 27)

In [ ]:
# check if operator name is the same
pd.set_option('display.max_colwidth',None)
matches_gdf[['COMPANY_NAME','Operator']].sample(10)

,COMPANY_NAME,Operator
86681,KP KAUFFMAN COMPANY INC,KP KAUFFMAN COMPANY INC
28022,NOBLE ENERGY INC,NOBLE ENERGY INC
234844,URSA OPERATING COMPANY LLC,TEP ROCKY MOUNTAIN LLC
120925,KP KAUFFMAN COMPANY INC,KP KAUFFMAN COMPANY INC
151143,NOBLE ENERGY INC,NOBLE ENERGY INC
41342,NOBLE ENERGY INC,NOBLE ENERGY INC
181647,CRESTONE PEAK RESOURCES OPERATING LLC,NOBLE MIDSTREAM SERVICES LLC
7358,NOBLE ENERGY INC,NOBLE ENERGY INC
87970,VERDAD RESOURCES LLC,NOBLE ENERGY INC
116794,CITATION OIL & GAS CORP,CITATION OIL & GAS CORP


In [ ]:
# Normalize the text in 'COMPANY_NAME' and 'Operator' columns before comparison
# This involves converting the text to lower case and stripping any leading/trailing spaces
validated_matches = matches_gdf[
    matches_gdf['COMPANY_NAME'].str.lower().str.strip() == matches_gdf['Operator'].str.lower().str.strip()
]

# Now, you can check the shape to see if there are any matches
print(validated_matches.shape)

(98663, 27)


In [ ]:
print(validated_matches[['COMPANY_NAME', 'Operator']].sample(10))

                                              COMPANY_NAME  \
34201   NOBLE ENERGY INC                                     
58470   CRESTONE PEAK RESOURCES OPERATING LLC                
6401    NOBLE ENERGY INC                                     
7081    BONANZA CREEK ENERGY OPERATING COMPANY LLC           
44870   CRESTONE PEAK RESOURCES OPERATING LLC                
78450   NOBLE ENERGY INC                                     
52948   CRESTONE PEAK RESOURCES OPERATING LLC                
73525   TIMBER CREEK OPERATING LLC                           
119281  OWN RESOURCES OPERATING LLC                          
14305   TIMBER CREEK OPERATING LLC                           

                                          Operator  
34201                             NOBLE ENERGY INC  
58470        CRESTONE PEAK RESOURCES OPERATING LLC  
6401                              NOBLE ENERGY INC  
7081    BONANZA CREEK ENERGY OPERATING COMPANY LLC  
44870        CRESTONE PEAK RESOURCES OPERATING LLC  

In [ ]:
# Merge the matches back to the original 'crudeoil_offlocation_gdf' to add 'flowlines_gdf' attributes where intersections occurred

# Dropp the 'index_right' column which is added by sjoin and duplicate geometry column
validated_matches.drop(columns=['index_right','geometry','Operator','Fluid','Material','Diam_in','Status','Length_ft','SHAPE_Length'], inplace=True)

/var/folders/p3/gs65vvgs1wd90mnxb3q3hvy80000gp/T/ipykernel_6211/847419937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validated_matches.drop(columns=['index_right','geometry','Operator','Fluid','Material','Diam_in','Status','Length_ft','SHAPE_Length'], inplace=True)


In [ ]:
# Merge based on index, ensuring all original entries in 'crudeoil_offlocation_gdf' are retained and adding 'flowlines_gdf' attributes where matches were found
all_flowlines = crudeoil_offlocation_gdf.merge(validated_matches, left_index=True, right_index=True, how="left")

In [ ]:
print(crudeoil_offlocation_gdf.shape)
print(flowlines_gdf.shape)
print(all_flowlines.shape)

(259979, 8)
(21942, 19)
(335177, 26)


In [ ]:
all_flowlines.head(2)

,Operator,Fluid,Material,Diam_in,Status,Length_ft,SHAPE_Length,geometry,LOCATION_ID,FLOWLINEID,STARTLOCATIONID,FLOWLINEACTION,ENTIRELINEREMOVED,ACTIONDESCRIPTION,RECEIVE_DATE,OPERATOR_NUM,COMPANY_NAME,LOCATIONTYPE,ENDLAT,ENDLONG,STARTLAT,STARTLONG,PIPEMATERIAL,BEDDINGMATERIAL,TYPEOFFLUIDTRANS,MAXOPPRESSURE
0,EVERGREEN NATURAL RESOURCES LLC,Gas,polly,4.0,Active,2277.71,693.972162,"MULTILINESTRING ((524642.670 4117088.796, 524635.855 4117065.943, 524642.839 4117043.875, 524644.806 4117038.110, 524644.948 4117037.721, 524647.344 4117029.801, 524649.373 4117015.664, 524647.124 4116999.342, 524639.679 4116984.231, 524630.942 4116970.047, 524621.881 4116956.759, 524614.674 4116944.822, 524608.274 4116932.252, 524601.572 4116917.128, 524593.134 4116899.610, 524582.780 4116881.528, 524570.950 4116863.161, 524560.121 4116845.209, 524550.495 4116828.320, 524541.644 4116811.514, 524532.448 4116793.800, 524522.199 4116774.155, 524511.641 4116754.211, 524501.932 4116736.341, 524493.554 4116719.288, 524485.915 4116703.239, 524478.660 4116688.610, 524472.318 4116676.228, 524466.204 4116663.878, 524461.745 4116654.085, 524460.993 4116652.434, 524460.799 4116652.268, 524459.589 4116649.783, 524455.570 4116641.354, 524449.475 4116628.485, 524442.644 4116614.823, 524434.908 4116600.482, 524425.811 4116585.828, 524414.601 4116570.923, 524399.704 4116555.207, 524380.607 4116537.113, 524372.634 4116530.176, 524364.897 4116524.118, 524357.109 4116516.189, 524345.951 4116505.112, 524333.116 4116492.371))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NOBLE ENERGY INC,Multiphase,Carbon Steel,3.0,Abandoned,651.58,198.525215,"MULTILINESTRING ((527997.281 4463899.920, 528172.790 4463992.704))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_flowlines.to_file('all_flowlines.geojson', driver='GeoJSON')